In [37]:
from datetime import datetime
import spotipy
import billboard
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [87]:
hit_data = pd.read_csv(r'C:\Users\eskil\PycharmProjects\tdt4173\Data\CSV-files\OLD_FILES\billboard-data.csv')
hit_data['name'] = hit_data['name'].str.lower()
#hit_data.head(30)
#hit_data.loc[hit_data['name'] == 'drunk face']

In [74]:
id = getTrackIdFromBillboard("drunk face", "Machine Gun Kelly")
#id = getTrackIdFromBillboard2("drunk face")
print(id)

3k0YJnqMKRZb8swo86vCkq


In [35]:
chart = billboard.ChartData('hot-100')
for entry in chart:
    print(entry.artist + "  :  " + entry.title)

24kGoldn Featuring iann dior  :  Mood
Ariana Grande  :  Positions
Drake Featuring Lil Durk  :  Laugh Now Cry Later
The Weeknd  :  Blinding Lights
Gabby Barrett Featuring Charlie Puth  :  I Hope
Jawsh 685 x Jason Derulo  :  Savage Love (Laxed - Siren Beat)
Internet Money & Gunna Featuring Don Toliver & NAV  :  Lemonade
Ariana Grande  :  34+35
Bad Bunny & Jhay Cortez  :  Dakiti
Cardi B Featuring Megan Thee Stallion  :  WAP
Justin Bieber Featuring Chance The Rapper  :  Holy
Pop Smoke Featuring Lil Baby & DaBaby  :  For The Night
DaBaby Featuring Roddy Ricch  :  Rockstar
Lewis Capaldi  :  Before You Go
Chris Brown & Young Thug  :  Go Crazy
BTS  :  Dynamite
Harry Styles  :  Watermelon Sugar
Luke Combs  :  Forever After All
Ava Max  :  Kings & Queens
Moneybagg Yo  :  Said Sum
Justin Bieber & benny blanco  :  Lonely
Kane Brown With Swae Lee & Khalid  :  Be Like That
Morgan Wallen  :  More Than My Hometown
Jack Harlow Featuring DaBaby, Tory Lanez & Lil Wayne  :  Whats Poppin
surf mesa Featurin

In [39]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [ ]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

In [40]:
def getTrackFeatures(id):
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      song_id = id
      name = meta['name']
      artist = meta['album']['artists'][0]['name']
      length = meta['duration_ms']
      popularity = meta['popularity']
      release_date = meta['album']['release_date']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']
      valence = features[0]['valence']
      mode = features[0]['mode']
      weeks = 1
      chart_date = 1
      target = 1

      track = {"name" : name, "artist" : artist, "song_id" : song_id, "length" : length, "popularity" : popularity, 
               "danceability" : danceability, "acousticness" : acousticness, "energy" : energy, "instrumentalness" : instrumentalness, 
               "liveness" : liveness, "loudness" : loudness, "speechiness" : speechiness, "tempo" : tempo, "time_signature" : time_signature, 
               "valence" : valence, "mode" : mode, "weeks" : weeks, "chart_date" : chart_date, "release_date" : release_date, "target" : target}
      return track

In [57]:
def getTrackIdFromBillboard(name, artist):
    results = sp.search(q="track:" + name + " artist:" + artist)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [76]:
def getTrackIdFromBillboard2(name):
    results = sp.search(q="track:" + name)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [88]:
def getChartDataFrame2(chart, fullDataFrame):
    tracks = []
    for entry in chart.entries:
        if (any (hit_data['name'] == entry.title.lower())):
            continue
        id = getTrackIdFromBillboard2(entry.title)
        if (id != "NONE"):
            #if (entry.artist in fullDataFrame.values) and (not any(dic['artist'] == entry.artist for dic in tracks)):
                #fullDataFrame.loc[fullDataFrame['artist'] == entry.artist, 'weeks'] += 1
            if (id not in fullDataFrame.values) and (entry.isNew):
                track = getTrackFeatures(id)
                track["chart_date"] = datetime.strptime(chart.date, '%Y-%m-%d').date()
                tracks.append(track)
    df = pd.DataFrame(tracks)
    fullDataFrame = fullDataFrame.append(df)
    return fullDataFrame

In [31]:
def getChartDataFrame(chart, fullDataFrame):
    tracks = []
    for entry in chart.entries:
        #if (any (hit_data['name'] == entry.title)):
            #continue
        id = getTrackIdFromBillboard(entry.title, entry.artist)
        if (id != "NONE"):
            if (entry.artist in fullDataFrame.values) and (not any(dic['artist'] == entry.artist for dic in tracks)):
                fullDataFrame.loc[fullDataFrame['artist'] == entry.artist, 'weeks'] += 1
            if (id not in fullDataFrame.values) and (entry.isNew):
                track = getTrackFeatures(id)
                track["chart_date"] = datetime.strptime(chart.date, '%Y-%m-%d').date()
                tracks.append(track)
    df = pd.DataFrame(tracks)
    fullDataFrame = fullDataFrame.append(df)
    return fullDataFrame

In [92]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    fullDataFrame = pd.DataFrame(columns = ['name', 'artist', 'song_id', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'mode', 'weeks', 'release_date', 'chart_date'])
    start = time.clock()
    while (chart.previousDate):
        #start = time.clock()
        chart = billboard.ChartData('hot-100', chart.previousDate)
        fullDataFrame = getChartDataFrame2(chart, fullDataFrame)
        #end = time.clock()
        #print("total time: " + str(end-start))
        i += 1
        if (i == 950):
            break
        if (i == 250):
            print("en firedel")
        if (i == 500):
            print("halveis")
        if (i == 750):
            print("tre firedel")
    end = time.clock()
    print("total time: " + str(end-start))
    return fullDataFrame

In [97]:
df.to_csv('flops_eskil')

In [93]:
df = gatherBillboardData()

C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """


en firedel
halveis
tre firedel
total time: 5988.7530541


C:\Users\eskil\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
